### 29. juni

Prøvde å gjenskape double descent grafen fra Nakkiran. Så at der hadde de gjenskapt grafer med CNN, og ikke f.eks RFF som brukt i Belkin. Koden kjørte i en evighet, må finnes en mer effektiv måte å gjenskape dem på? 

Klarer ikke finne igjen koden jeg brukte til dette nå som jeg samler alt sammen, må ha overskrevet den med en annen kode jeg prøvde for å få frem double descend ved å implementere label noise og plotte test error vs epochs. Denne koden vises nedenfor. Den tar også alt for lang tid å kjøre så jeg har ikke fått testet om den funker som den skal.

Hørte at vi kanskje skulle sammenligne tre ulike metoder i slutten av prosjektet, hvor back-propagation er en av dem. Brukte derfor litt tid på å lese mer mer opp i dybden på hvordan metoden funker mens koden for double descent kjørte.

Så på parameterendringer ved å endre på størrelsen på nettverket i koden fra 23-24_06_23.ipynb og 28_06_23.ipynb:
Å minke batch size gjør at modellen raskere blir accurate og loss blir mindre for CNN, samme skjedde for MLP. Dersom batch_size=1 blir det mange fluktuasjoner i accuracy (epochs=10) men jevnt over høy accuracy. For MLP med flere nevroner i de skjulte lagene minker vektendringen. Vektendringen i CNN øker med minkende batch size.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import trange

In [ ]:
# Define the width parameter
width = 64

In [ ]:
# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(784, width)
        self.layer2 = nn.Linear(width, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        return x

In [ ]:
# Define the training parameters
batch_size = 128
learning_rate = 0.01
num_epochs = 1000

In [ ]:
# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model, loss function, and optimizer
model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
test_errors = []
for epoch in trange(num_epochs):
    total_loss = 0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Compute test error
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = 100 * correct / total
    test_error = 100 - test_accuracy
    test_errors.append(test_error)

    # Print progress
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {total_loss:.4f}, Test Error: {test_error:.2f}%')


In [ ]:
# Plot test error versus epochs
epochs = range(1, num_epochs)
plt.plot(epochs, test_errors)
plt.xlabel('Epochs')
plt.ylabel('Test Error (%)')
plt.title(f'Double Descent Curve for Width={width}')
plt.grid(True)
plt.show()

### Backpropagation

Backpropagation er en algoritme som brukes for å trene nevrale nettverk. Algoritmen utfører en backward pass fra output til input og justerer modellens parametre for å minke feilen og øke nøyaktigheten til nettverket. Vi antar at vi har et MLP-nettverk med 784 input-noder, to skjulte lag med 16 noder og et output-lag med 10 noder.

Siden Loss-funksjonen finner gjennomsnittlig loss per bilde for flere tusenvis av treningsbilder vil hvordan vi justerer vektene og biasene til ett gradient descent steg avhenge av alle bildene. For å forklare backpropagation fokuserer vi først på kun ett bilde. 

Anta at vi sender inn et bilde av et 2-tall og at nettverket ikke er trent enda. Output-nodene vil derfor gi tilfeldige verdier mellom 0 og 1. Vi kan ikke direkte endre disse output-verdiene da vi kun kan påvirke vektene og biasene. Vi ønsker å øke verdien til den tredje noden (3. siden 0-9) og minke verdien til alle de andre nodene, for om nettverket er fullstendig trent skal den tredje noden ha verdi 1 og resten av nodene skal ha verdi 0. Størrelsen av endringen bør være proporsjonal til hvor langt unna hver nodeverdi er fra den idelle verdien. Husk at hver output-verdi er en vektet sum av alle verdiene i det forrige laget bluss bias som i likningen for vektet sum. For å øke verdien av den tredje noden kan man fra likningen se at man kan øke bias $b$ eller vekten $w_i$. Vektene har ulik påvirkning, hvor vektene fra nodene med størst verdi i forrige lag har størst påvirkning på loss-funksjonen (kan sammenlignes med Hebbs teori i nevrovitenskap). Man bør øke vektene $w_i$ proporsjonalt til nodeverdiene $a_i$ i det forrige laget og proporsjonalt med hvor mye de trenger å endres. Siden man ønsker å minke verdien til de andre nodene i output-laget må man endre vektene til hver av disse tilsvarende. Dette forteller oss dermed hva som må endres i det nest-siste laget og hvordan for å få minst mulig loss. Man får en liste av ønskede endringer og gjør samme prosess for bias og vekter i det siste laget for å finne ønskede endringer til dette laget. Husk at dette kun er hvordan vektene og biasene bør være dersom vi sender inn et bilde av et 2-tall. Man gjør nøyaktig det samme for mange andre bilder for å trene nettverket til å gjenkjenne flere ulike bilder. De gjennomsnittlige endringenetil hver vekt og bias er den negative gradienten til loss-funksjonen.

Siden det er ekstremt tidkrevende å finne endringen til hvert eneste treningsbilde og hvert gradient descent steg deler vi datasettet inn i batches og regner ut gradient descent steget til hver batch som gjør prosessen raskere. Denne prosessen kalles stochastic gradient descent.

### Deep double descent
Deep double descent er et fenomen som oppstår i CNNs hvor prestasjonen først øker, så blir verre og deretter øker igjen med økende modellstørrelse, datastørrelse eller treningstid. Dette fenomenet unngås ofte ved nøye regulering. Man er enda ikke helt sikker på hvorfor det skjer.

Double descent-fenomenet ble først oppdaget av Mikhail Belkin som undersøkte påstanden mange hadde om at "større modeller alltid er bedre", selv om statisktisk ML-teori forutsa at større modeller hadde større sannsynlighet for overfitting. 

Bias-variance tradeoff er en egenskap til ML-modeller hvor variansen til et sett med data kan reduseres ved å øke bias. Man ønsker å minimere begge disse kildene til feil. En bias feil er en feil som fører til underfitting, og variansefeil fører til overfitting pga mye tilfeldig støy i treningsdataen.

Data- og label-støy er antatte avvik fra det faktiske datasettet. Datastøy er avvik i dataen, altså bildene, mens labelstøy er avvik i labels.